In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-pricing-dataset/house_prices.csv


In [52]:
# Load the dataset
file_path = "/kaggle/input/house-pricing-dataset/house_prices.csv"
df = pd.read_csv(file_path)

# Display basic info to confirm successful loading
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  object 
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  object 
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

(None,
            id             date     price  bedrooms  bathrooms  sqft_living  \
 0  7129300520  20141013T000000  221900.0         3       1.00         1180   
 1  6414100192  20141209T000000  538000.0         3       2.25         2570   
 2  5631500400  20150225T000000  180000.0         2       1.00          770   
 3  2487200875  20141209T000000  604000.0         4       3.00         1960   
 4  1954400510  20150218T000000  510000.0         3       2.00         1680   
 
    sqft_lot  floors waterfront  view  ... grade  sqft_above  sqft_basement  \
 0      5650     1.0          N     0  ...     7        1180              0   
 1      7242     2.0          N     0  ...     7        2170            400   
 2     10000     1.0          N     0  ...     6         770              0   
 3      5000     1.0          N     0  ...     7        1050            910   
 4      8080     1.0          N     0  ...     8        1680              0   
 
    yr_built  yr_renovated  zipcode      

In [53]:
# Drop unnecessary columns
df_cleaned = df.drop(columns=["id", "date"])

# Encode categorical variables
df_cleaned["waterfront"] = df_cleaned["waterfront"].map({"Y": 1, "N": 0})
df_cleaned["condition"] = LabelEncoder().fit_transform(df_cleaned["condition"])

# Define features and target variable
X = df_cleaned.drop(columns=["price"])
y = df_cleaned["price"]

**Pre-Processing**

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Drop unnecessary columns
df_cleaned = df.drop(columns=["id", "date"])

# Encode categorical variables
df_cleaned["waterfront"] = df_cleaned["waterfront"].map({"Y": 1, "N": 0})
df_cleaned["condition"] = LabelEncoder().fit_transform(df_cleaned["condition"])

# Define features and target variable
X = df_cleaned.drop(columns=["price"])
y = df_cleaned["price"]

# Split into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add constant term for statsmodels
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

# Fit model with all variables (baseline model)
model_all = sm.OLS(y_train, X_train_sm).fit()

# Store results
model_results = {"All Variables": model_all}

# Show summary of the baseline model
model_all.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     2363.
Date:                Mon, 10 Mar 2025   Prob (F-statistic):               0.00
Time:                        17:14:38   Log-Likelihood:            -2.3542e+05
No. Observations:               17290   AIC:                         4.709e+05
Df Residuals:                   17272   BIC:                         4.710e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          6.219e+06   3.24e+06      1.917      0.055    -1.4e+05    1.26e+07
bedrooms      -3.421e+04   2082.322    -16.428      0.000   -3.83e+04   -3.01e+04
bathrooms      4.439e+04   3607.091     12.306      0.000    3.73e+04    5.15e+04
sqft_living     108.9036      2.530     43.042      0.000     103.944     113.863
sqft_lot          0.0835      0.058      1.445      0.148      -0.030       0.197
floors         7148.8493   3958.506      1.806      0.071    -610.224    1.49e+04
waterfront     5.615e+05   1.95e+04     28.786      0.000    5.23e+05       6e+05
view           5.359e+04   2358.736     22.718      0.000     4.9e+04    5.82e+04
condition      1.367e+04   1357.528     10.070      0.000     1.1e+04    1.63e+04
grade          9.512e+04   2371.099     40.116      0.000    9.05e+04    9.98e+04
sqft_above       69.9982      2.486     28.162      0.000      65.126      74.870
sqft_basement    38.8995      2.938     13.238      0.000      33.140      44.659
yr_built      -2642.7084     80.986    -32.632      0.000   -2801.449   -2483.968
yr_renovated     21.4308      4.071      5.264      0.000      13.451      29.411
zipcode        -548.1478     36.460    -15.034      0.000    -619.614    -476.682
lat            5.969e+05   1.19e+04     50.305      0.000    5.74e+05     6.2e+05
long          -1.944e+05   1.46e+04    -13.312      0.000   -2.23e+05   -1.66e+05
sqft_living15    20.9021      3.809      5.488      0.000      13.436      28.368
sqft_lot15       -0.3246      0.082     -3.968      0.000      -0.485      -0.164
==============================================================================
Omnibus:                    14975.304   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1712544.182
Skew:                           3.632   Prob(JB):                         0.00
Kurtosis:                      51.212   Cond. No.                     9.57e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.91e-20. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Drop unnecessary columns
df_cleaned = df.drop(columns=["id", "date"])

# Encode categorical variables
df_cleaned["waterfront"] = df_cleaned["waterfront"].map({"Y": 1, "N": 0})
df_cleaned["condition"] = LabelEncoder().fit_transform(df_cleaned["condition"])

# Define features and target variable
X = df_cleaned.drop(columns=["price"])
y = df_cleaned["price"]

# Split into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add constant term for statsmodels
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

# Fit model with all variables (baseline model)
model_all = sm.OLS(y_train, X_train_sm).fit()

# Store results
model_results = {"All Variables": model_all}

# Show summary of the baseline model
model_all.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     2363.
Date:                Mon, 10 Mar 2025   Prob (F-statistic):               0.00
Time:                        17:14:38   Log-Likelihood:            -2.3542e+05
No. Observations:               17290   AIC:                         4.709e+05
Df Residuals:                   17272   BIC:                         4.710e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          6.219e+06   3.24e+06      1.917      0.055    -1.4e+05    1.26e+07
bedrooms      -3.421e+04   2082.322    -16.428      0.000   -3.83e+04   -3.01e+04
bathrooms      4.439e+04   3607.091     12.306      0.000    3.73e+04    5.15e+04
sqft_living     108.9036      2.530     43.042      0.000     103.944     113.863
sqft_lot          0.0835      0.058      1.445      0.148      -0.030       0.197
floors         7148.8493   3958.506      1.806      0.071    -610.224    1.49e+04
waterfront     5.615e+05   1.95e+04     28.786      0.000    5.23e+05       6e+05
view           5.359e+04   2358.736     22.718      0.000     4.9e+04    5.82e+04
condition      1.367e+04   1357.528     10.070      0.000     1.1e+04    1.63e+04
grade          9.512e+04   2371.099     40.116      0.000    9.05e+04    9.98e+04
sqft_above       69.9982      2.486     28.162      0.000      65.126      74.870
sqft_basement    38.8995      2.938     13.238      0.000      33.140      44.659
yr_built      -2642.7084     80.986    -32.632      0.000   -2801.449   -2483.968
yr_renovated     21.4308      4.071      5.264      0.000      13.451      29.411
zipcode        -548.1478     36.460    -15.034      0.000    -619.614    -476.682
lat            5.969e+05   1.19e+04     50.305      0.000    5.74e+05     6.2e+05
long          -1.944e+05   1.46e+04    -13.312      0.000   -2.23e+05   -1.66e+05
sqft_living15    20.9021      3.809      5.488      0.000      13.436      28.368
sqft_lot15       -0.3246      0.082     -3.968      0.000      -0.485      -0.164
==============================================================================
Omnibus:                    14975.304   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1712544.182
Skew:                           3.632   Prob(JB):                         0.00
Kurtosis:                      51.212   Cond. No.                     9.57e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.91e-20. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

**Backward Elimination**

In [56]:
# Backward Elimination: Start with all features and remove least significant one by one

def backward_elimination(X, y, significance_level=0.05):
    """ Performs backward elimination based on p-values """
    X = sm.add_constant(X)  # Add constant for intercept
    model = sm.OLS(y, X).fit()
    
    while True:
        p_values = model.pvalues
        max_p_value = p_values.max()  # Find max p-value
        if max_p_value > significance_level:  # If above threshold, remove feature
            worst_feature = p_values.idxmax()
            X = X.drop(columns=[worst_feature])
            model = sm.OLS(y, X).fit()
        else:
            break
            
    return model, X.columns

# Apply backward elimination
model_backward, selected_features_backward = backward_elimination(X_train, y_train)

# Store results
model_results["Backward Elimination"] = model_backward

# Display selected features
selected_features_backward


Index(['bedrooms', 'bathrooms', 'sqft_living', 'waterfront', 'view',
       'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15',
       'sqft_lot15'],
      dtype='object')

**Backward Elimination Results:**
The following features were retained:
bedrooms, bathrooms, sqft_living, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated, zipcode, lat, long, sqft_living15, sqft_lot15
sqft_lot and floors were removed due to high p-values.

**Forward Selection**

In [57]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def forward_selection(X, y, significance_level=0.05):
    """ Performs forward selection based on p-values """
    selected_features = []
    remaining_features = list(X.columns)
    X_const = sm.add_constant(X)  # Add constant for intercept
    best_model = None

    while remaining_features:
        best_p_value = float('inf')
        best_feature = None
        temp_model = None

        for feature in remaining_features:
            temp_features = selected_features + [feature]
            temp_X = X_const[temp_features + ['const']]
            model = sm.OLS(y, temp_X).fit()
            p_value = model.pvalues[feature]

            if p_value < significance_level and p_value < best_p_value:
                best_p_value = p_value
                best_feature = feature
                temp_model = model

        if best_feature is not None:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            best_model = temp_model
        else:
            break

    return best_model, selected_features

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply forward selection
model_forward, selected_features_forward = forward_selection(X_train, y_train)

# Display selected features
print("Selected Features:", selected_features_forward)
print(model_forward.summary())


Selected Features: ['bedrooms', 'bathrooms', 'sqft_living', 'yr_built', 'grade', 'lat', 'waterfront', 'view', 'zipcode', 'long', 'condition', 'sqft_above', 'sqft_basement', 'yr_renovated', 'sqft_living15', 'sqft_lot15']
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     2677.
Date:                Mon, 10 Mar 2025   Prob (F-statistic):               0.00
Time:                        17:14:39   Log-Likelihood:            -2.3542e+05
No. Observations:               17290   AIC:                         4.709e+05
Df Residuals:                   17274   BIC:                         4.710e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                

**Forward Selection Results:**
The following features were selected:
bedrooms, bathrooms, sqft_living, yr_built, grade, lat, waterfront, view, zipcode, long, condition, sqft_above, sqft_basement, yr_renovated, sqft_living15, sqft_lot15
sqft_lot and floors were not selected, similar to Backward Elimination.

**Stepwise Selection**

In [58]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def stepwise_selection(X, y, significance_level=0.05):
    """ Performs stepwise selection (both forward and backward) """
    selected_features = []
    remaining_features = list(X.columns)
    X_const = sm.add_constant(X)  # Add constant for intercept

    while remaining_features:
        best_p_value = float('inf')
        best_feature = None

        # Forward Step: Find best feature to add
        for feature in remaining_features:
            temp_features = selected_features + [feature]
            temp_X = X_const[temp_features + ['const']]
            model = sm.OLS(y, temp_X).fit()
            p_value = model.pvalues[feature]

            if p_value < significance_level and p_value < best_p_value:
                best_p_value = p_value
                best_feature = feature

        if best_feature is not None:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)

        # Backward Step: Remove worst feature if necessary
        temp_X = X_const[selected_features + ['const']]
        model = sm.OLS(y, temp_X).fit()
        p_values = model.pvalues.drop('const')  # Exclude intercept
        worst_p_value = p_values.max()

        if worst_p_value > significance_level:
            worst_feature = p_values.idxmax()
            selected_features.remove(worst_feature)

        if best_feature is None and worst_p_value <= significance_level:
            break

    return model, selected_features


# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply stepwise selection
model_stepwise, selected_features_stepwise = stepwise_selection(X_train, y_train)

# Display selected features
print("Selected Features:", selected_features_stepwise)
print(model_stepwise.summary())


Selected Features: ['bedrooms', 'bathrooms', 'sqft_living', 'yr_built', 'grade', 'lat', 'waterfront', 'view', 'zipcode', 'long', 'condition', 'sqft_above', 'sqft_basement', 'yr_renovated', 'sqft_living15', 'sqft_lot15']
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     2677.
Date:                Mon, 10 Mar 2025   Prob (F-statistic):               0.00
Time:                        17:14:40   Log-Likelihood:            -2.3542e+05
No. Observations:               17290   AIC:                         4.709e+05
Df Residuals:                   17274   BIC:                         4.710e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                